In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import style
style.use('ggplot')
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv("IMDB Dataset.csv")
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [3]:
def no_of_words(text):
    try:
        words=text.split()
        word_count=len(words)
    except:
        print("exception")
        word_count=0
    return word_count  

In [4]:
def no_of_sentences(text):
    try:
        sentences = sent_tokenize(text)
        number_of_sentences = len(sentences)
    except:
        print("exception")
        number_of_sentences=0
    return number_of_sentences

In [5]:
df['word count']=df['review'].apply(no_of_words)

In [6]:
df['sentence count']=df['review'].apply(no_of_sentences)

In [7]:
df.head(10)

,review,sentiment,word count,sentence count
0,One of the other reviewers has mentioned that ...,positive,307,10
1,A wonderful little production. <br /><br />The...,positive,162,7
2,I thought this was a wonderful way to spend ti...,positive,166,4
3,Basically there's a family where a little boy ...,negative,138,6
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,230,9
5,"Probably my all-time favorite movie, a story o...",positive,119,6
6,I sure would like to see a resurrection of a u...,positive,150,1
7,"This show was an amazing, fresh & innovative i...",negative,174,9
8,Encouraged by the positive comments about this...,negative,130,7
9,If you like original gut wrenching laughter yo...,positive,33,3


In [8]:
df.drop_duplicates(inplace = True)

In [9]:
stop = stopwords.words('english')
wl = WordNetLemmatizer()

In [10]:
mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot",
           "'cause": "because", "could've": "could have", "couldn't": "could not",
           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
           "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will",
           "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
           "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
           "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have",
           "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
           "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
           "might've": "might have","mightn't": "might not","mightn't've": "might not have",
           "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
           "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
           "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
           "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
           "she's": "she is", "should've": "should have", "shouldn't": "should not",
           "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is",
           "that'd": "that would", "that'd've": "that would have", "that's": "that is",
           "there'd": "there would", "there'd've": "there would have", "there's": "there is",
           "here's": "here is","they'd": "they would", "they'd've": "they would have",
           "they'll": "they will", "they'll've": "they will have", "they're": "they are",
           "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
           "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
           "we're": "we are", "we've": "we have", "weren't": "were not",
           "what'll": "what will", "what'll've": "what will have","what're": "what are",
           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have",
           "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
           "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
           "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
           "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
           "y'all're": "you all are","y'all've": "you all have","you'd": "you would",
           "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
           "you're": "you are", "you've": "you have" }

In [11]:
def clean_text(text,lemmatize = True):
    soup = BeautifulSoup(text, "html.parser") #remove html tags
    text = soup.get_text()
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    text = re.sub(r'\.(?=\S)', '. ',text) #add space after full stop
    text = re.sub(r'http\S+', '', text) #remove urls
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #remove punctuation
    #tokens = re.split('\W+', text) #create tokens
    if lemmatize:
        text = " ".join([wl.lemmatize(word) for word in text.split() if word not in stop and word.isalpha()]) #lemmatize
    else:
        text = " ".join([word for word in text.split() if word not in stop and word.isalpha()])
    return text

In [12]:
df_copy = df.copy()

In [13]:
df['review']=df['review'].apply(clean_text,lemmatize = True)

/var/folders/5b/cmxfzbyx2bl_h9g6lj3qcjhc0000gn/T/ipykernel_45834/53484750.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser") #remove html tags


In [14]:
df.sentiment = [ 1 if each == "positive" else 0 for each in df.sentiment]     

In [15]:
df['word count']=df['review'].apply(no_of_words)
df['sentence count']=df['review'].apply(no_of_sentences)

In [16]:
df.head()

,review,sentiment,word count,sentence count
0,one reviewer mentioned watching oz episode hoo...,1,164,1
1,wonderful little production filming technique ...,1,84,1
2,thought wonderful way spend time hot summer we...,1,84,1
3,basically family little boy jake think zombie ...,0,64,1
4,petter matteis love time money visually stunni...,1,125,1


In [17]:
word_count_1=df['word count']//3

In [18]:
df['word_count_1']=word_count_1
df.head()

,review,sentiment,word count,sentence count,word_count_1
0,one reviewer mentioned watching oz episode hoo...,1,164,1,54
1,wonderful little production filming technique ...,1,84,1,28
2,thought wonderful way spend time hot summer we...,1,84,1,28
3,basically family little boy jake think zombie ...,0,64,1,21
4,petter matteis love time money visually stunni...,1,125,1,41


In [19]:
def process_review(row):
    review = row['review']
    word_count = row['word_count_1']
    words = word_tokenize(review)
    few_words = words[:word_count]
    shortened_review = " ".join(few_words)
    return shortened_review

In [20]:
df['shortened_review'] = df.apply(process_review, axis=1)

In [21]:
df_1 = df[['shortened_review', 'sentiment']]

In [22]:
df_1.head()

,shortened_review,sentiment
0,one reviewer mentioned watching oz episode hoo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake think zombie ...,0
4,petter matteis love time money visually stunni...,1


In [23]:
df_1['word count2']=df['shortened_review'].apply(no_of_words)

/var/folders/5b/cmxfzbyx2bl_h9g6lj3qcjhc0000gn/T/ipykernel_45834/2271361747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['word count2']=df['shortened_review'].apply(no_of_words)


In [24]:
df_1.head()

,shortened_review,sentiment,word count2
0,one reviewer mentioned watching oz episode hoo...,1,54
1,wonderful little production filming technique ...,1,28
2,thought wonderful way spend time hot summer we...,1,28
3,basically family little boy jake think zombie ...,0,21
4,petter matteis love time money visually stunni...,1,41


In [25]:
df_1.to_csv('first_part.csv')

In [26]:
def process_review2(row):
    review = row['review']
    word_count = row['word_count_1']
    words = word_tokenize(review)
    word_count_total=row['word count']
    few_words = words[word_count:word_count+word_count+1]
    shortened_review = " ".join(few_words)
    return shortened_review

In [27]:
df['shortened_review2'] = df.apply(process_review2, axis=1)

In [28]:
df_2 = df[['shortened_review2', 'sentiment']]

In [29]:
df_2['word count2']=df['shortened_review2'].apply(no_of_words)

/var/folders/5b/cmxfzbyx2bl_h9g6lj3qcjhc0000gn/T/ipykernel_45834/2853341932.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['word count2']=df['shortened_review2'].apply(no_of_words)


In [30]:
df_2.head()

,shortened_review2,sentiment,word count2
0,cell glass front face inwards privacy high age...,1,55
1,seamless editing guided reference williams dia...,1,29
2,disappointed realize match point risk addictio...,1,29
3,zombie ok first going make film must decide th...,0,22
4,character meet connect one connected one way a...,1,42


In [31]:
df_2.to_csv('second_part.csv')

In [32]:
def process_review3(row):
    review = row['review']
    word_count = row['word_count_1']
    words = word_tokenize(review)
    few_words = words[-word_count:]
    shortened_review = " ".join(few_words)
    return shortened_review

In [33]:
df['shortened_review3'] = df.apply(process_review3, axis=1)

In [34]:
df_3 = df[['shortened_review3', 'sentiment']]

In [35]:
df_3['word count3']=df['shortened_review3'].apply(no_of_words)

/var/folders/5b/cmxfzbyx2bl_h9g6lj3qcjhc0000gn/T/ipykernel_45834/90137878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['word count3']=df['shortened_review3'].apply(no_of_words)


In [36]:
df_3.head()

,shortened_review3,sentiment,word count3
0,saw struck nasty surreal could say ready watch...,1,54
1,guard rather use traditional dream technique r...,1,28
2,impressed scarlet johanson managed tone sexy i...,1,28
3,ruin film expected see boogeyman similar movie...,0,21
4,human relation find sincere fulfillment one di...,1,41


In [37]:
df_3.to_csv('third_part.csv')